## Robot setup

In [135]:
import numpy as np
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("..")
sys.path.append("../../")

from armControl import *
from forwardKinematics import robot_config
from utils.motor_setup import maxonGearSmall
import armControl

from utils.NatNetClient2 import NatNetClient
from utils.GetJointData import data, NatNetFuncs #receiveNewFrame, receiveRigidBodyFrameList

import signal

import matplotlib.pyplot as plt

socket_ip = '192.168.0.113'
socket_port = 1125

server_ip = "192.168.0.105"
multicastAddress = "239.255.42.99"

joint_names = ['Base','RigidBody']
ids = [0, 1]

#trajPlanner = trajectoryGenerator()

motors = maxonGearSmall()
motors.tcp_init(socket_ip, socket_port)

#axis_motor_indexes = np.array([-1, -1, -1, 0, 3, 2, 1]) #indexed from 0 for motors and axis
#velocity = np.ones(8)*3.14/20


Opening socket at ip: 192.168.0.113 using port: 1125


## Command motors w/ indexing for joints

def commandJoints(setpoint_arm, trajectory = True):
    myArm = armControl.remoteRobotArm()
    myArm.jointAngleSetpoint = setpoint_arm
    myArm.updateMotorArmMixing()
    setpoint_motor = np.zeros(8)
    setpoint_motor[axis_motor_indexes[3:7]] = myArm.motorAngleSetpoint[3:7]
    if trajectory:
        motors.run_trajectory(setpoint_motor, velocity)
    else:
        motors.command_motors_radians(setpoint_motor)


## Arm Motors

In [136]:
print("Arming motors now...")
motors.arm_motors()

Arming motors now...
initializing motors to [0 0 0 0 0 0 0 0]
*** C side has an error or needs to be armed ***

Arming motors


## Manual motor zeroing

In [137]:
from importlib import reload
reload(armControl)

myArm = armControl.remoteRobotArm()

setpoint_arm = np.array([0.0,0.0,-0.,0,0.0,0,0])
myArm.commandJoints(motors, setpoint_arm)
time.sleep(0.5)


In [138]:
for i in range(8):
    motors.zero_motors_radians(i, motors.get_motors_position_radians()[i])

## Test arm joint mixing

### this is old, now in "commandJoints" above

import armControl
from importlib import reload
reload(armControl)

setpoint_arm = np.array([0,0,0,0,0,0,-0])
myArm = armControl.remoteRobotArm()
myArm.jointAngleSetpoint = setpoint_arm
myArm.updateMotorArmMixing()
print(myArm.motorAngleSetpoint)

setpoint_motor = np.zeros(8)
setpoint_motor[axis_motor_indexes[3:7]] = myArm.motorAngleSetpoint[3:7]
print(setpoint_motor)
motors.run_trajectory(setpoint_motor, velocity)


In [2]:
print(myArm.jointUpperLimits)
print(myArm.jointLowerLimits)
a = np.linspace(myArm.jointLowerLimits[-1],myArm.jointUpperLimits[-1], num=50)
b = np.linspace(myArm.jointLowerLimits[-2],myArm.jointUpperLimits[-2], num=500)
c = np.linspace(myArm.jointLowerLimits[-3],myArm.jointUpperLimits[-3], num=50)
d = np.linspace(myArm.jointLowerLimits[-4],myArm.jointUpperLimits[-4], num=50)

print(a)
print(b)
#c = np.meshgrid(a,b)
#print(c)

setpoint_arm = np.array([0,0,0,0.,0,0.,0.0])
myArm.commandJoints(motors, setpoint_arm)

time.sleep(5)

#Tracking class
print("Starting streaming client now...")
streamingClient = NatNetClient(server_ip, multicastAddress, verbose = print_trak_data)
NatNet = NatNetFuncs()
streamingClient.newFrameListener = NatNet.receiveNewFrame
streamingClient.rigidBodyListListener = NatNet.receiveRigidBodyFrameList

prev_frame = 0
time.sleep(0.5)

streamingClient.run()

time.sleep(0.5)

track_data = data(joint_names,ids)

time.sleep(0.5)

track_data.parse_data(NatNet.joint_data, NatNet.frame) #updates the frame and data that is being used

pos_list = []
ang_list = []
rot_list = []
for m in range(3):
    pos_sweep_list = []
    ang_sweep_list = []
    for k in range(len(d)):
        #time.sleep(5)
        for j in range(len(c)):
            #time.sleep(5)
            setpoint_arm = np.array([0,0,0,d[k],c[j],b[i],0])
            myArm.commandJoints(motors, setpoint_arm, trajectory = True)
            time.sleep(5)
            for i in range(len(b)):
                setpoint_arm = np.array([0,0,0,d[k],c[j],b[i],0])
                myArm.commandJoints(motors, setpoint_arm, trajectory = False)
                time.sleep(0.02)
                #add in optitrack position measurements
                base = track_data.bodies[0].homogenous_mat
                base_inv = track_data.bodies[0].homg_inv
                needle = track_data.bodies[1].homogenous_mat
                needle_base, needlebase_pos, needlebase_euler, _ = track_data.homg_mat_mult(base_inv,needle)
                needlebase_deg = np.array(needlebase_euler) * 180 / np.pi
                needlebase_pos_mm = np.array(needlebase_pos)*1000
                needle_base_rot = needle_base[0:3,0:3]
                pos_sweep_list.append(needlebase_pos_mm)
                ang_sweep_list.append(needlebase_euler)
                rot_sweep_list.append(needle_base_rot)
            time.sleep(5)
    pos_list.append(pos_sweep_list)
    ang_list.append(ang_sweep_list) #eulers 
    rot_list.append(rot_sweep_list) #rotation matrix

sweep1_pos = pos_list[0]
sweep2_pos = pos_list[1]
sweep3_pos = pos_list[2]

sweep1_ang = ang_list[0]
sweep2_ang = ang_list[1]
sweep3_ang = ang_list[2]

sweep1_rot = rot_list[0]
sweep2_rot = rot_list[1]
sweep3_rot = rot_list[2]

#plot x,y,z

for i in range(3):
    
    figure()
    plt.plot(sweep1_pos[:][i], label='Sweep 1') 
    plt.plot(sweep2_pos[:][i], label='Sweep 2') 
    plt.plot(sweep3_pos[:][i]), label='Sweep 3')
    plt.ylabel('Position')
    plt.legend()
    plt.show()
    
    figure()
    plt.plot(sweep1_ang[:][i], label='Sweep 1') 
    plt.plot(sweep2_ang[:][i], label='Sweep 2') 
    plt.plot(sweep3_ang[:][i]), label='Sweep 3')
    plt.ylabel('Rotation')
    plt.legend()
    plt.show()



for k in range(len(sweep1_pos))
    #error in position
    error_pos_12[k] = abs(sweep1_pos[k]-sweep2_pos[k])
    error_pos_13[k] = abs(sweep1_pos[k]-sweep3_pos[k])
    error_pos_23[k] = abs(sweep2_pos[k]-sweep3_pos[k])
    
for i in range(len(sweep1_ang))
    #invert rotation matrices 
    sweep1_rot_inv[i] = sweep1_rot[i].T
    sweep2_rot_inv[i] = sweep2_rot[i].T
    #error in rotation matrices
    error_rot_12[i] = sweep1_rot_inv[i]*sweep2_rot[i]
    error_rot_13[i] = sweep1_rot_inv[i]*sweep3_rot[i]
    error_rot_23[i] = sweep2_rot_inv[i]*sweep3_rot[i]
    #convert error in rotation matrices to eulers
    error_euler_12[i] = mat2euler(error_ang_12[i])
    error_euler_13[i] = mat2euler(error_ang_13[i])
    error_euler_23[i] = mat2euler(error_ang_23[i])
    
print("Sweep 1 & 2 Position Error: {}, Sweep 1 & 3 Position Error: {}, Sweep 2 & 3 Position Error: {}" .format(error_pos_12, error_pos_13, error_pos_23))
print("Sweep 1 & 2 Angle Error: {}, Sweep 1 & 3 Angle Error: {}, Sweep 2 & 3 Angle Error: {}" .format(error_euler_12, error_euler_13, error_euler_23))  
    
  

    
    


setpoint_arm = np.array([0,0,0,0.,0,0.,0.0])
myArm.commandJoints(motors, setpoint_arm)

SyntaxError: invalid syntax (<ipython-input-2-2463c653ead1>, line 90)

## Sine wave / profile test

In [ ]:
dt = 0.005
runtime = 10
num_elements = int(runtime/dt)
setpoints = np.zeros((num_elements, 7))
frequency = 0.1
amplitude = np.pi/6
for i in range(num_elements):
    setpoints[i,:] = (np.ones(7) * np.sin(i*dt*2*pi*frequency)*amplitude)


start_time = time.time()

currents_mA = []
for i in range(num_elements):
    current_time = time.time()
    setpoint = setpoints[i]
    myArm.commandJoints(motors, setpoint, trajectory=False)
    time.sleep(dt)

time.sleep(1.5)

motors.run_trajectory(np.zeros(8), velocity)


## Cleanup

In [134]:
setpoints = np.zeros(8)
velocity = np.ones(8)*3.14/5
motors.run_trajectory(setpoints, velocity)
time.sleep(20)
motors.tcp_close()

In [120]:
motors.tcp_close()